In [157]:
#!pip install porter2stemmer
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
#from porter2stemmer import Porter2Stemmer
import re

nltk.download('stopwords')
stopwordList = stopwords.words('english')

nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Subir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Subir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Subir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [158]:
df = pd.read_csv('pnlp_data_en.csv', delimiter=';')
df.dropna()
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_colwidth', 50)

# rename columns for easier usability
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
# preview
df.head(10)

,report_grouping,question_text,comments
0,Large Department,Please tell us what is working well.,"we do what our customers need, we communicate ..."
1,Large Department,Please tell us what is working well.,Customs business development continues to grow...
2,Large Department,Please tell us what is working well.,"I think the team work hard, are committed to c..."
3,Large Department,Please tell us what is working well.,Overall working towards a customer centric env...
4,Large Department,Please tell us what is working well.,Customer centricity is a growing culture in th...
5,Large Department,Please tell us what is working well.,•Develop a comfortable rapport with clients an...
6,Large Department,Please tell us what is working well.,THE CUSTOMER IS THE CENTER.
7,Large Department,Please tell us what is working well.,"we are usually seeking customer satisfaction, ..."
8,Large Department,Please tell us what is working well.,Alignment between regional office and country ...
9,Large Department,Please tell us what is working well.,"innovation, customer relations ship and custom..."


In [159]:
# Apostrophe Dictionary
apostrophe = {
    "aren't" : "are not",
    "can't" : "cannot",
    "couldn't" : "could not",
    "didn't" : "did not",
    "doesn't" : "does not",
    "don't" : "do not",
    "hadn't" : "had not",
    "hasn't" : "has not",
    "haven't" : "have not",
    "he'd" : "he would",
    "he'll" : "he will",
    "he's" : "he is",
    "i'd" : "I would",
    "i'd" : "I had",
    "i'll" : "I will",
    "i'm" : "I am",
    "isn't" : "is not",
    "it's" : "it is",
    "it'll":"it will",
    "i've" : "I have",
    "let's" : "let us",
    "mightn't" : "might not",
    "mustn't" : "must not",
    "shan't" : "shall not",
    "she'd" : "she would",
    "she'll" : "she will",
    "she's" : "she is",
    "shouldn't" : "should not",
    "that's" : "that is",
    "there's" : "there is",
    "they'd" : "they would",
    "they'll" : "they will",
    "they're" : "they are",
    "they've" : "they have",
    "we'd" : "we would",
    "we're" : "we are",
    "weren't" : "were not",
    "we've" : "we have",
    "what'll" : "what will",
    "what're" : "what are",
    "what's" : "what is",
    "what've" : "what have",
    "where's" : "where is",
    "who'd" : "who would",
    "who'll" : "who will",
    "who're" : "who are",
    "who's" : "who is",
    "who've" : "who have",
    "won't" : "will not",
    "wouldn't" : "would not",
    "you'd" : "you would",
    "you'll" : "you will",
    "you're" : "you are",
    "you've" : "you have",
    "'re": " are",
    "wasn't": "was not",
    "we'll":" will",
    "didn't": "did not",
    "'s": "is",
    "'re": "are"
}

short_words = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

def apos_short_dict(text, dictionary):
    for word in text.split():
        if word.lower() in dictionary:
            if word.lower() in text.split():
                text = text.replace(word, dictionary[word.lower()])
    return text

In [160]:
def remove_html(txt):
    '''Remove HTML'''
    txt = BeautifulSoup(txt, 'lxml')
    return txt.get_text()

def remove_punctuation(surveyText):
    return "".join([i for i in surveyText if i not in string.punctuation])

def remove_stopwords(surveyText):
    return [w for w in surveyText if w not in stopwordList]

def word_lemmatizer(surveyText):
    lemmatizer = WordNetLemmatizer()
    w_tokenizer = WhitespaceTokenizer()
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(surveyText)]

def word_stemmer(surveyText):
    stemmer = PorterStemmer()
    return [stemmer.stem(i) for i in surveyText]
    
def preprocessing(txt, punctuation= False, tokenize= False, stopwords= False, correct_apos= False, 
                  shortWords= False, specialCharacter= False, numbers= False, singleChar= False,
                 lematization= False, stemming= False):
    
    cleanedTxt = txt.apply(lambda x: remove_html(x))
    
    if punctuation:
        cleanedTxt = cleanedTxt.apply(lambda x:remove_punctuation(x))
        
    if tokenize:
        cleanedTxt = cleanedTxt.apply(lambda x:word_tokenize(x.lower()))
        
    if stopwords:
        cleanedTxt = cleanedTxt.apply(lambda x: remove_stopwords(x))
        
    if correct_apos:
        cleanedTxt = cleanedTxt.apply(lambda x: apos_short_dict(str(x),apostrophe))
        
    if shortWords:
        cleanedTxt = cleanedTxt.apply(lambda x: apos_short_dict(str(x),short_words))
    
    if specialCharacter:
        '''Replacing Special Characters with space'''
        cleanedTxt = cleanedTxt.apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',str(x)))
    
    if numbers:
        '''Replacing Numbers with space'''
        cleanedTxt = cleanedTxt.apply(lambda x: re.sub(r'[^a-zA-Z]',' ',x))
        
    if singleChar:
        '''Removing words whom length is one'''
        cleanedTxt = cleanedTxt.apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
    
    if lematization:
        cleanedTxt = cleanedTxt.apply(lambda x: word_lemmatizer(x))
        
    if stemming:
        cleanedTxt = cleanedTxt.apply(lambda x: word_stemmer(x))
        #pass
    
    return cleanedTxt
      
df['comments'] = preprocessing(df['comments'], punctuation= True, tokenize= True, stopwords= True, 
                                correct_apos= True, shortWords= True, specialCharacter= True, 
                               numbers= True, singleChar= True, lematization= True, stemming= True)
df.head(5)

,report_grouping,question_text,comments
0,Large Department,Please tell us what is working well.,"[custom, need, commun, aperiod]"
1,Large Department,Please tell us what is working well.,"[custom, busi, develop, continu, grow, expand,..."
2,Large Department,Please tell us what is working well.,"[think, team, work, hard, commit, continu, imp..."
3,Large Department,Please tell us what is working well.,"[overal, work, toward, custom, centric, enviro..."
4,Large Department,Please tell us what is working well.,"[custom, centric, grow, cultur, compani, creat..."


In [161]:
# de-tokenization
detokenizer = []
for i in range(len(df)):
    t = ' '.join(df['comments'][i]) # tokenized text
    detokenizer.append(t)

df['comments'] = detokenizer
df.head(5)

,report_grouping,question_text,comments
0,Large Department,Please tell us what is working well.,custom need commun aperiod
1,Large Department,Please tell us what is working well.,custom busi develop continu grow expand use in...
2,Large Department,Please tell us what is working well.,think team work hard commit continu improv goo...
3,Large Department,Please tell us what is working well.,overal work toward custom centric environ work...
4,Large Department,Please tell us what is working well.,custom centric grow cultur compani creat posit...
